In [51]:
import numpy as np
import pandas as pd
import ast
from sentence_transformers import util
import matplotlib.pyplot as plt
from itertools import chain
from sentence_transformers import SentenceTransformer, util
import openai


In [52]:
expunations = pd.read_csv('expunations_complete_with_score_and_generated_pairs.csv', encoding='utf-8')


In [53]:
expunations

,ID,explanation,keywords,joke,funniness_average,filtered_funniness_average,expression1,expression2,sentence1,sentence2
0,hom_657,['the phrase dish it out means to criticizie a...,"[[""waitress's"", 'dish it out'], [''], [''], ['...",She was only a Waitress ' s daughter but she s...,0.6,1.000000,dish it out,dish it out,The waitress had to dish it out to all the cus...,"During the feedback session, she was able to d..."
1,het_1635,"['', 'This is a play on words. The words ""pane...","[[''], ['glazier', 'window', 'feel your pane',...",When the glazier was sent to the hospital room...,1.2,2.000000,pane,pain,The glazier carefully installed the new pane i...,The patient experienced severe pain after the ...
2,hom_1510,"['sandpaper is normally rough on one side ', '...","[['inventor', 'sandpaper', 'rough'], ['invento...",The inventor of sandpaper had a rough time of ...,1.6,1.600000,rough,rough,He had a rough day dealing with the challenges.,The sandpaper is rough and scratches the surface.
3,hom_955,"['', '', '', 'A contortionist is someone who c...","[[''], [''], [''], ['contortionists', 'die', '...","OLD CONTORTIONISTS never die , they just meet ...",0.2,1.000000,meet their end,meet their end,Everyone will meet their end eventually.,The contortionist could effortlessly meet thei...
4,hom_1505,"['', 'This is a play on words. The word ""party...","[[''], ['whose place to have a party at', 'par...",When you can ' t decide with your friends whos...,0.4,0.666667,party,party,We are hosting a party at our home this Saturday.,The Liberal party announced its new policy age...
...,...,...,...,...,...,...,...,...,...,...
1885,hom_2076,"['', ""This is a pun on the phrase 'stroke of g...","[[''], ['Da Vinci', 'stroke', 'genius'], ['Leo...",Leonardo Da Vinci could use a paint brush to m...,0.4,0.500000,stroke of genius,stroke of genius,"Her presentation was truly a stroke of genius,...","Using a fine brush, the artist made a delicate..."
1886,hom_1437,['driven can mean to drive a car or it can me ...,"[['cars', 'driven'], ['cars', 'driven', 'eden'...",The earliest mention of cars is in the Bible w...,1.0,1.250000,driven out,driven out,The manager was driven out of the company afte...,He was driven out of town in his new sports car.
1887,het_1530,"['son sounds like sun ', ""This is a pun on 'su...","[['doting', 'son'], ['parents', 'son', 'worshi...",Some doting parents are son worshipers .,0.8,1.000000,son,sun,She is proud of her son for winning the compet...,The sun shines brightly during the summer months.
1888,het_100,"['lion sounds like line ', ""This is a pun on t...","[['zoo', 'lion'], ['phone', 'zoo', 'lion'], ['...",I phoned the zoo but the lion was busy .,1.6,1.600000,lion,line,The lion roamed freely in the zoo.,The phone line was busy during the call.


# Calculate embeddings

### MODEL OpenAi text-embedding-3-small




In [56]:
# Set your OpenAI API key
# openai.api_key = ...

def sentence2embedding_openai(text, model="text-embedding-3-small"):
    """https://www.linkedin.com/news/story/european-ai-gets-260bn-boost-6316004/
    Fetch the embedding for a given text using OpenAI's updated embedding API.
    """
    response = openai.embeddings.create(
        input=text,
        model=model
    )
    # Access the embeddings from the response's 'data' attribute
    return response.data[0].embedding
    #return np.array(response['data'][0]['embedding'])

### MODEL all-MiniLM-L6-v2

In [57]:
def sentence2embedding_minilm(sentence):

    # Load the all-MiniLM-L6-v2 model
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    return model.encode(sentence, convert_to_tensor=True)

    

### Test: the the models return different results

In [59]:

sentence1 = "The quick brown fox jumps over the lazy dog."
sentence2 = "jumps"

embedding_sentence1 = sentence2embedding_minilm(sentence1)
embedding_sentence2 = sentence2embedding_minilm(sentence2)
similarity = util.cos_sim(embedding_sentence1, embedding_sentence2)
print(f" Model MiniLM all-MiniLM-L6-v2 - Cosine Similarity between '{sentence1}' and '{sentence2}': {similarity.item():.4f}")

embedding_sentence1 = sentence2embedding_openai(sentence1)
embedding_sentence2 = sentence2embedding_openai(sentence2)
similarity = util.cos_sim(embedding_sentence1, embedding_sentence2)
print(f" Model OpenAi text-embedding-3-small - Cosine Similarity between '{sentence1}' and '{sentence2}': {similarity.item():.4f}")



 Model MiniLM all-MiniLM-L6-v2 - Cosine Similarity between 'The quick brown fox jumps over the lazy dog.' and 'jumps': 0.3710
 Model OpenAi text-embedding-3-small - Cosine Similarity between 'The quick brown fox jumps over the lazy dog.' and 'jumps': 0.4372


### Calculate embeddings OpenAI

In [60]:
expunations['emb_openai_joke'] = None  # Initialize column with None
expunations['emb_openai_joke'] = expunations['emb_openai_joke'].astype(object)  # Ensure it's object type

expunations['emb_openai_expression1'] = None  # Initialize column with None
expunations['emb_openai_expression1'] = expunations['emb_openai_expression1'].astype(object)  # Ensure it's object type

expunations['emb_openai_expression2'] = None  # Initialize column with None
expunations['emb_openai_expression2'] = expunations['emb_openai_expression2'].astype(object)  # Ensure it's object type

expunations['emb_openai_sentence1'] = None  # Initialize column with None
expunations['emb_openai_sentence1'] = expunations['emb_openai_sentence1'].astype(object)  # Ensure it's object type

expunations['emb_openai_sentence2'] = None  # Initialize column with None
expunations['emb_openai_sentence2'] = expunations['emb_openai_sentence2'].astype(object)  # Ensure it's object type


## skipped 237

for i in range (len(expunations)):
    joke=expunations['joke'][i]
    expunations.at[i, 'emb_openai_joke'] = sentence2embedding_openai(joke)

    expression1=expunations['expression1'][i]
    expunations.at[i, 'emb_openai_expression1'] = sentence2embedding_openai(expression1)

    expression2=expunations['expression2'][i]
    if expression2 != expression1:
        expunations.at[i, 'emb_openai_expression2'] = sentence2embedding_openai(expression2)
    else:
        expunations.at[i, 'emb_openai_expression2'] = sentence2embedding_openai(expression1)

    sentence1=expunations['sentence1'][i]
    expunations.at[i, 'emb_openai_sentence1'] = sentence2embedding_openai(sentence1)

    sentence2=expunations['sentence2'][i]
    expunations.at[i, 'emb_openai_sentence2'] = sentence2embedding_openai(sentence2)

    print (i, joke)


0 She was only a Waitress ' s daughter but she sure could dish it out .
1 When the glazier was sent to the hospital room to check the cracked window , he told the patient in the body cast , ' ' I ' ve come to feel your pane . ' '
2 The inventor of sandpaper had a rough time of it .
3 OLD CONTORTIONISTS never die , they just meet their end .
4 When you can ' t decide with your friends whose place to have a party at it ' s known as party politics .
5 Back when the news was written on clay tablets there was a lot of breaking news .
6 My friend came around for dinner . We ate for 20 minutes , then he fainted again .
7 A band of teenagers cruised past my neighbor ' s house and pelted it with rotten tomatoes . Police described it as a drive - by fruiting .
8 None but the brave deserve the fair .
9 Romance isn ' t a science - it ' s a heart .
10 OLD STATUES never die they just get busted .
11 Actors are cast through their good and bad stages .
12 Patience is a virtue heavy in wait
13 OLD POLI

### Calculate embeddings MiniLM

In [64]:
expunations['emb_minilm_joke'] = None  # Initialize column with None
expunations['emb_minilm_joke'] = expunations['emb_minilm_joke'].astype(object)  # Ensure it's object type

expunations['emb_minilm_expression1'] = None  # Initialize column with None
expunations['emb_minilm_expression1'] = expunations['emb_minilm_expression1'].astype(object)  # Ensure it's object type

expunations['emb_minilm_expression2'] = None  # Initialize column with None
expunations['emb_minilm_expression2'] = expunations['emb_minilm_expression2'].astype(object)  # Ensure it's object type

expunations['emb_minilm_sentence1'] = None  # Initialize column with None
expunations['emb_minilm_sentence1'] = expunations['emb_minilm_sentence1'].astype(object)  # Ensure it's object type

expunations['emb_minilm_sentence2'] = None  # Initialize column with None
expunations['emb_minilm_sentence2'] = expunations['emb_minilm_sentence2'].astype(object)  # Ensure it's object type


for i in range (len(expunations)):
    joke=expunations['joke'][i]
    expunations.at[i, 'emb_minilm_joke'] = sentence2embedding_minilm(joke).tolist()

    expression1=expunations['expression1'][i]
    expunations.at[i, 'emb_minilm_expression1'] = sentence2embedding_minilm(expression1).tolist()

    expression2=expunations['expression2'][i]
    if expression2 != expression1:
        expunations.at[i, 'emb_minilm_expression2'] = sentence2embedding_minilm(expression2).tolist()
    else:
        expunations.at[i, 'emb_minilm_expression2'] = sentence2embedding_minilm(expression1).tolist()

    sentence1=expunations['sentence1'][i]
    expunations.at[i, 'emb_minilm_sentence1'] = sentence2embedding_minilm(sentence1).tolist()

    sentence2=expunations['sentence2'][i]
    expunations.at[i, 'emb_minilm_sentence2'] = sentence2embedding_minilm(sentence2).tolist()

    print (i, joke)


0 She was only a Waitress ' s daughter but she sure could dish it out .
1 When the glazier was sent to the hospital room to check the cracked window , he told the patient in the body cast , ' ' I ' ve come to feel your pane . ' '
2 The inventor of sandpaper had a rough time of it .
3 OLD CONTORTIONISTS never die , they just meet their end .
4 When you can ' t decide with your friends whose place to have a party at it ' s known as party politics .
5 Back when the news was written on clay tablets there was a lot of breaking news .
6 My friend came around for dinner . We ate for 20 minutes , then he fainted again .
7 A band of teenagers cruised past my neighbor ' s house and pelted it with rotten tomatoes . Police described it as a drive - by fruiting .
8 None but the brave deserve the fair .
9 Romance isn ' t a science - it ' s a heart .
10 OLD STATUES never die they just get busted .
11 Actors are cast through their good and bad stages .
12 Patience is a virtue heavy in wait
13 OLD POLI

In [65]:
expunations

,ID,explanation,keywords,joke,funniness_average,filtered_funniness_average,expression1,expression2,sentence1,sentence2,emb_openai_joke,emb_openai_expression1,emb_openai_expression2,emb_openai_sentence1,emb_openai_sentence2,emb_minilm_joke,emb_minilm_expression1,emb_minilm_expression2,emb_minilm_sentence1,emb_minilm_sentence2
0,hom_657,['the phrase dish it out means to criticizie a...,"[[""waitress's"", 'dish it out'], [''], [''], ['...",She was only a Waitress ' s daughter but she s...,0.6,1.000000,dish it out,dish it out,The waitress had to dish it out to all the cus...,"During the feedback session, she was able to d...","[0.030972516164183617, -0.01517175231128931, -...","[0.014902290888130665, -0.03285151720046997, -...","[0.014902290888130665, -0.03285151720046997, -...","[0.005138686392456293, -0.007233967538923025, ...","[0.010029598139226437, -0.009848949499428272, ...","[-0.03195299953222275, -0.005314176436513662, ...","[0.02359960786998272, 0.10548564046621323, -0....","[0.02359960786998272, 0.10548564046621323, -0....","[-0.03609524294734001, 0.055144697427749634, 0...","[0.0027361176908016205, 0.02424769103527069, -..."
1,het_1635,"['', 'This is a play on words. The words ""pane...","[[''], ['glazier', 'window', 'feel your pane',...",When the glazier was sent to the hospital room...,1.2,2.000000,pane,pain,The glazier carefully installed the new pane i...,The patient experienced severe pain after the ...,"[-0.01962309144437313, 0.04074713587760925, -0...","[0.027517514303326607, -0.026580538600683212, ...","[0.014019208028912544, -0.012372410856187344, ...","[-0.03295952454209328, 0.04716017097234726, -0...","[-0.03114989958703518, 0.02944694459438324, -0...","[-0.07717575877904892, 0.03188902512192726, 0....","[-0.008362654596567154, 0.006861681118607521, ...","[-0.06583624333143234, 0.005977192427963018, 0...","[0.038147103041410446, -0.005319848190993071, ...","[0.011634974740445614, 0.03996971994638443, -0..."
2,hom_1510,"['sandpaper is normally rough on one side ', '...","[['inventor', 'sandpaper', 'rough'], ['invento...",The inventor of sandpaper had a rough time of ...,1.6,1.600000,rough,rough,He had a rough day dealing with the challenges.,The sandpaper is rough and scratches the surface.,"[0.01849677972495556, 0.016096102073788643, 0....","[-0.02635161764919758, 0.004799611866474152, 0...","[-0.02635161764919758, 0.004799611866474152, 0...","[-0.026946933940052986, -0.001050555962137878,...","[0.004222436808049679, 0.02058914303779602, 0....","[-0.028803806751966476, 0.06886464357376099, 0...","[-0.020329659804701805, -0.012983807362616062,...","[-0.020329659804701805, -0.012983807362616062,...","[0.04728022962808609, 0.13506457209587097, 0.0...","[-0.010853069834411144, -0.0011316132731735706..."
3,hom_955,"['', '', '', 'A contortionist is someone who c...","[[''], [''], [''], ['contortionists', 'die', '...","OLD CONTORTIONISTS never die , they just meet ...",0.2,1.000000,meet their end,meet their end,Everyone will meet their end eventually.,The contortionist could effortlessly meet thei...,"[0.05470442399382591, 0.028842754662036896, 0....","[0.046937935054302216, 0.020673619583249092, 0...","[0.046937935054302216, 0.020673619583249092, 0...","[0.02289336919784546, 0.019332490861415863, 0....","[0.04577306658029556, 0.008717942051589489, 0....","[-0.008412803523242474, -0.04027313366532326, ...","[-0.04347410425543785, 0.022733362391591072, 0...","[-0.04347410425543785, 0.022733362391591072, 0...","[-0.022082282230257988, -0.03496827557682991, ...","[-0.05765708535909653, 0.005961173679679632, -..."
4,hom_1505,"['', 'This is a play on words. The word ""party...","[[''], ['whose place to have a party at', 'par...",When you can ' t decide with your friends whos...,0.4,0.666667,party,party,We are hosting a party at our home this Saturday.,The Liberal party announced its new policy age...,"[0.020545732229948044, -0.01821220852434635, 0...","[0.016281891614198685, -0.004896087571978569, ...","[0.01626163721084

In [66]:
expunations.to_csv("expunations_complete_with_score_generated_pairs_minilm_openai_embeddings.csv", encoding='utf-8', index=False)